# Arbres de décision

## Comment construit-on un arbre de décision ?
Le principal algorithme utilisé pour l’apprentissage d’arbres et l’algorithme CART (Classification And Regression Tree). Cet algorithme permet d’apprendre aussi bien des arbres de classification que des arbres de régression. 
1. Pour chaque nœud de l’arbre et si le critère d’arrêt n’y est pas rempli, on cherche la variable et le seuil permettant de séparer le plus efficacement possible la population du nœud courant (gain d’information maximal), le critère d’information étant différent en régression (MSE, MAE) et en classification (Gini, entropie). La feature (on utilise le plus souvent qu’une feature) utilisée pour le split est celle apportant la plus grande amélioration du critère de split.
2. On répète 1. sur les deux nœuds-fils obtenus.

Lorsqu’une observation est classée dans une feuille donnée :
* En régression, la valeur retournée correspondra à la moyenne des valeurs de y pour les observations du training set ayant été groupés dans cette feuille à l’entrainement (on montre que c’est pour la moyenne que l’erreur est minimisée au sein de chaque feuille). On remarque qu’autoriser des feuilles à très peu de membre à l’entrainement peut contribuer à une variance élevée en prédiction. Corolaire : imposer une taille minimale de feuille à l’entrainement a un smoothing effect.
* En classification, le label retourné sera celui le plus représenté parmi ceux des observations du training set ayant été groupés dans cette feuille à l’entrainement.

## Quels algorithmes de construction d’arbre connaissez-vous ? 
L’algorithme CART est le plus utilisé et est notamment implémenté dans scikit-learn. Un autre algorithme connu est le C4.5/C5.

## Quels sont les principaux inconvénients des arbres de décision ? Que peut-on faire pour palier à ces inconvénients ?
Le premier inconvénient des arbres est qu’ils sont assez prompts à overfitter les données. Le risque est d’autant plus grand lorsque notre nombre d’observations n’est pas beaucoup plus élevé que le nombre de features. Plusieurs méthodes sont disponibles pour contrôler l’apprentissage d’un arbre :
* On peut jouer sur le critère d’arrêt. On peut ainsi subordonner le split d’un nœud à un ou plusieurs critères :  nombre minimal d’observations pour splitter le nœud, nombre minimal d’observations pour constituer une feuille terminale, profondeur maximale, nombre maximal de feuilles terminales, amélioration minimale du critère de split (A utiliser avec prudence, cette stratégie peut se révéler myope : un split relativement médiocre avec une faible amélioration du critère peut être suivi de splits très intéressants).
* On peut laisser l’arbre croitre au maximum et l’élaguer (prune) à postériori. A l’élagage de l’arbre correspond un algorithme dédié. L’élagage va consister à trouver l’arbre réalisant le meilleur compromis entre minimisation d’un critère d’information (MSE, etc.) et complexité de l’arbre (ex : son nombre total de feuilles terminales). Le pruning n’est actuellement pas supporté par scikit-learn.
* Dans les deux cas, le choix du ou des meilleurs paramètres de régularisation et de l’arbre associé se fait par cross-validation (dans le second cas, il s’agit du paramètre donnant une plus ou moins grande importance à la complexité du modèle). 
* Dans le cas d’un problème de haute dimension, on peut faire précéder l’entrainement d’une étape de réduction de dimension (PCA, feature sélection, etc.).

Un second inconvénient est que de par (le caractère hiérarchique de) l’algorithme les construisant, les arbres sont très sensibles aux variations des données (une erreur, un changement au niveau d’un nœud se propage à tous les nœuds suivants) et donc instables : de petites différences dans le training set peuvent conduire à un ensemble de splits / à des arbres très différents. Ce problème peut être atténué par l’utilisation de multiples arbres dans une méthode d’ensemble (le bagging moyenne différents arbres et atténue cette source de variance).

Un troisième inconvénient des arbres utilisés seuls est qu’ils sont limités dans la complexité qu’ils peuvent apprendre. Certaines fonctions ou concepts leurs sont très difficiles à apprendre. Pour certaines fonctions ils apparaissent largement sous-optimaux (ex : frontière de décision à 45°). Là encore, leur utilisation dans des méthodes d’ensemble mène à de bien meilleurs résultats.

Un quatrième inconvénient est qu’en présence d’une classe majoritaire dans le dataset, les arbres obtenus sont biaisés en faveur de la classe majoritaire. C’est un problème plus large qui n’affecte pas que les arbres de décision. Dans leur cas cela provient du fait que les critères de Gini / d’entropie sont conçus pour être maximums si un nœud présente la même distribution que le dataset d’entrée qui par défaut est supposé équilibré (Gini/entropie maximale à 0 .5). Si on souhaite (sans ré échantillonner) obtenir les meilleurs splits dans un dataset déséquilibré (c’est-à-dire qui crée des nœuds de répartition éloignée de celle du dataset de départ), il faut pondérer les critères de split (afin qu’ils soient maximaux à 0.9 par exemple). Trois solutions finalement sont envisageables :
* Rééquilibrer le dataset par up- ou downsampling de la classe minoritaire ou majoritaire respectivement avant de lancer l’apprentissage.
* Surpondérer la classe minoritaire (on introduit en fait des pondérations dans les critères de split) de façon à ce que le dataset pondéré apparaisse équilibré.
* Utiliser la balanced accuracy plutôt que l’accuracy qui perd beaucoup de son utilité sur les datasets déséquilibrés (un modèle affectant toujours une observation à la classe majoritaire peut obtenir une bonne accuracy) qui correspond à la moyenne pondérée des accuracies sur chaque classe (il existe plein de définitions équivalentes comme par exemple balanced accuracy = moyenne des recalls par classe). Elle est comprise entre 0 et 1. Si le dataset est équilibré alors balanced accuracy et accuracy coïncident. Si le classificateur performe de la même façon sur chaque classe alors balanced accuracy et accuracy coïncident.

On peut ajouter que les surfaces de prédiction (régression) / les frontières de décision (classification) sont très irrégulières (lack of smoothness), ce qui peut être un problème notamment en régression où on cherche le plus souvent à approcher des fonctions relativement régulières. 

## Quels sont les principaux avantages des arbres de décision ?
Le principal avantage des arbres de décision est qu’ils sont particulièrement transparents, la prédiction est facile à comprendre. C’est le modèle offrant la plus grande explicabilité au métier (d’autant qu’ils peuvent s’interpréter comme un exemple de règle métiers) bien qu’un arbre profond puisse devenir plus difficile à interpréter. On peut nuancer davantage : en cas de multicolinéarité, commenter le choix de features fait à la construction de l’arbre (par exemple pour en dériver des règles métier) est plus délicat puisque pour deux features colinéaires, chacune peut arbitrairement être choisie par l’arbre pour l’information qu’elle apporte. L’application d’une PCA avant entrainement peut répondre à ce problème, toutefois les nouvelles features obtenues souffrent souvent d’être difficilement interprétables.

Le second avantage des arbres de décision est qu’ils demandent peu de préprocessing des données d’entrée. En particulier, ils n’exigent pas que les données soient normalisées. 

Un troisième avantage des arbres de décision est qu’ils supportent les données mixtes. Le split se base sur une mesure d’impureté dans les branches qu’il crée et non sur les valeurs de la variable. Le split sur une variable catégorielle prendrait la forme de l’appartenance à certaines classes. Le split sur une variable continue prendrait la forme d’un seuil. Tout du moins en théorie : l’implémentation des arbres de décision de scikit-learn ne supporte que les données continues. On est obligé d’en passer par un OneHotEncoder au prix d’un peu plus de calcul.

On peut ajouter qu’ils sont également relativement peu sensibles aux outliers. 

## Quelle est la complexité de la construction d’un arbre ? 
Soit n le nombre d’observations du training set et p le nombre de features, le coût de la construction d’un arbre se situe entre O(p.n.log(n)) et O(p².n.log(n)). Cf. https://scikit-learn.org/stable/modules/tree.html#complexity

## Quels critères de split connaissez-vous ? Pouvez-vous expliquer brièvement chacun d’entre eux ?
En régression, les critères de split les plus communément utilisés sont la MSE et la MAE.
En classification, les critères de Gini et de (cross-)entropie sont les plus utilisés. Ces critères sont construits de sorte à être pour un nœud donné, minimums quand le nœud est pur et maximums lorsque les classes des observations s’y distribuent de la même manière que dans le dataset (qui est souvent présupposé équilibré). Dans le cas de la classification binaire d’un dataset équilibré, ce maximum est atteint lorsque la proportion de chaque classe au sein de chaque nœud est de ½. 